In [70]:
## ADVENT OF CODE 2022, Day 11
## Edmund Dickinson, Python implementation

# File read
input_folder = "Input/"
input_file = "day11a.txt"
file_path = input_folder + input_file

with open(file_path) as file:
    input = file.read()

In [71]:
# Class definition: Item
# Define a worry level and a holding monkey
class Item:
    def __init__(self, worry, holdmonkey):
        self.worry = worry
        self.monkey = holdmonkey

# Class definition: Monkey
class Monkey:
    def __init__(self, monkeydef, itemlist):
        self.inspectcount = 0
        self.itemlist = itemlist
        # Store greatest common divisor of the set of monkeys
        self.gcd = 1

        # Process the monkey definition string
        # Line 0: Monkey [id]:
        # Line 1: Starting items: [items]
        # Line 2: Operation: new = old [oper_type] [oper_val]
        # Line 3: Test: divisible by [divisor]
        # Line 4: If true: throw to monkey [truetarget_id]
        # Line 5: If false: throw to monkey [falsetarget_id]

        self.id = int(monkeydef[0].split()[-1].replace(":",""))
        
        # Comma-separated list of items, trim preamble and split to list               
        item_str = monkeydef[1].replace("Starting items: ","").split(",")
        items = [int(item) for item in item_str]

        # Create a new item for every one initially held and define as being held by this monkey
        for item in items:
            self.itemlist.append(Item(item, self.id))
        
        self.oper_type = monkeydef[2].split()[-2]
        oper_val = monkeydef[2].split()[-1]

        # Handle case of squaring, defined as "old * old"
        if (oper_val == "old"):
            self.oper_type = "^"
            # Only allow squaring, oper_val will be ignored
            self.oper_val = 0
        else:
            # Standard multiplication or addition, cast to int
            self.oper_val = int(oper_val)

        self.divisor = int(monkeydef[3].split()[-1])

        self.truetarget_id = int(monkeydef[4].split()[-1])
        self.falsetarget_id = int(monkeydef[5].split()[-1])

    def inspectAll(self, calming=True):
        # Inspect the current list of items held by this monkey
        # Take copy as some items will be removed
        for idx, item in enumerate(self.itemlist):
            # Check which items are held by current monkey
            if (item.monkey == self.id):
                # Accumulator of inspection count
                self.inspectcount += 1
                
                worry = item.worry
                monkey = item.monkey

                # Perform operation on worry
                # Store worry as remainder with respect to multiple of greatest common divisor
                # w = (n*gcd) + x
                # w + a = (n*gcd) + (x+a)
                # w * a = (a*n*gcd) + (x*a)
                # w * w = ((n*gcd) + 2x)*n*gcd + x * x

                if (self.oper_type == "+"):
                    worry = worry + self.oper_val
                elif (self.oper_type == "*"):            
                    worry = worry * self.oper_val
                elif (self.oper_type == "^"):
                    worry = worry * worry
                
                # Divisor
                if(calming):
                    worry = worry // 3
                else:
                    worry = worry % self.gcd

                # Check condition and throw item
                if ((worry % self.divisor) == 0):
                    monkey = self.truetarget_id
                else:
                    monkey = self.falsetarget_id
                
                # Update holding monkey to "throw" item
                self.itemlist[idx].worry = worry
                self.itemlist[idx].monkey = monkey

In [72]:
# File parsing
# Split each monkey
monkey_str = input.split("\n\n")
monkeys = []
items = []

for item in monkey_str:
    # Assume monkeys are listed in order, so order of monkeys list is same as their ids
    # Add a monkey to the list
    monkeys.append(Monkey(item.splitlines(), items))

# Compute greatest common divisor
gcd = 1
for monkey in monkeys:
    gcd = gcd * monkey.divisor

# Set greatest common divisor
for monkey in monkeys:
    monkey.gcd = gcd

In [74]:
# Analysis
# Each monkey takes one turn each round
num_rounds = 10000
for round in range(num_rounds):
    for monkey in monkeys:
        monkey.inspectAll(calming = False)

# Return inspection counts and sort in descending order
inspectcounts = [monkey.inspectcount for monkey in monkeys]
inspectcounts.sort(reverse = True)

monkey_business = (inspectcounts[0] * inspectcounts[1])
print("Level of monkey business:", monkey_business)

Level of monkey business: 19309892877
